# sp3-orbital
Her finner du noen funksjoner som kan brukes til å plotte løsninger av Schrödingerligningen for hydrogenliknende orbitaler. Her skal vi bare visualisere noen sp3-orbitaler. Denne notebook'en inneholder ikke forklaringer.

## Kode for plotting av orbitaler:

In [ ]:
from scipy.special import sph_harm  # Sfæriske harmoniske funksjoner
from scipy.special import genlaguerre  # Generaliserte Laguerre polynomer
from math import factorial  # Funksjon for å regne ut fakultet
import numpy as np  # Bibliotek for å kunne jobbe med numeriske lister
import pyvista as pv  # For 3D-plotting
pv.set_plot_theme('document')

In [ ]:
def radiell(n, l, r):
    """Beregn radiell del av bølgefunksjonen
    
    Parametere
    ----------
    n : heltall, dette er hovedkvantetallet.
    l : heltall, dette er vinkelkvantetallet.
    r : tall, detter er posisjonen vi evaluerer funksjonen i
    
    Resultat
    --------
    ut : tall, verdien for radiell del i angitt punkt.
    
    """
    pre = np.sqrt((2 / n)**3 * factorial(n - l - 1) / (2 * n * factorial(n + l)))
    r_hat = 2 * r / n
    laguerre = genlaguerre(n - l - 1, 2*l + 1)
    return pre * np.exp(-r_hat / 2) * r_hat**l * laguerre(r_hat)


def vinkelavhengighet(l, m, theta, phi):
    """Regn ut vinkelavhengighet for bølgefunksjonen.
    
    Her tar vi også hensyn til komplekse tall og gjør løsningene
    reelle.
    
    Parametere
    ----------
    l : heltall, vinkelkvantetallet.
    m : heltall, magnetisk kvantetall.
    theta : tall, polarvinkel (engelsk: polar angle).
    phi : tall, asimut (engelsk: azimuthal angle)
    
    Returnerer
    ----------
    ut : tall, verdi for vinkelavhengigheten til bølgefunksjonen.
    
    """
    # SciPy vil ha:
    # - m som første argument, l som andre
    # - asimut som første vinkel (kaller den "theta" i dokumentasjonen)
    # - polar som andre vinkel (kaller den "phi" i dokumentasjonen)
    if m == 0:
        vinkel = sph_harm(m, l, phi, theta)
    elif m < 0:
        vinkel = sph_harm(m, l, phi, theta) - (-1)**m * sph_harm(-m, l, phi, theta)
        vinkel = vinkel * (1j / np.sqrt(2))
    elif m > 0:
        vinkel = sph_harm(-m, l, phi, theta) + (-1)**m * sph_harm(m, l, phi, theta)
        vinkel = vinkel * (1 / np.sqrt(2))
    return np.real(vinkel)

def beregn_orbital(n, l, m, r, theta, phi):
    # Sjekk at kvantetall er gyldig:
    if n < 1:
        raise ValueError(f'Ugyldig "n={n}". n = 1, 2, 3, ...')
    if l < 0 or l > n - 1:
        raise ValueError(f'Ugyldig "l={l}", l = 0, 1, ..., n-1')
    if m > l or m < -l:
        raise ValueError(f'Ugyldig "m={m}", m = -l, -l + 1, ..., 0, ..., l - 1, l')
    return radiell(n, l, r) *  vinkelavhengighet(l, m, theta, phi)

In [ ]:
def plot_hjelp_orbital(r, theta, phi, psi, cmap='viridis'):
    """Lag et 3D plot ved å bruke PyVista. Overflaten fargelegges etter verdiene til psi."""
    xyz = np.array(
        [
            r * np.sin(theta) * np.cos(phi),
            r * np.sin(theta) * np.sin(phi),
            r * np.cos(theta),
        ]
    )
    orbital = np.abs(psi)**2
    X, Y, Z = orbital * xyz
    grid = pv.StructuredGrid(X, Y, Z)
    plotter = pv.Plotter(notebook=True)
    plotter.set_background('white')
    fortegn = 2. * (psi - psi.min()) / np.ptp(psi) - 1
    plotter.add_mesh(grid, scalars=fortegn.T, show_scalar_bar=True, cmap=cmap, clim=[-1, 1],
                     scalar_bar_args={'title': 'Skalert fortegn', 'color': 'k'})
    plotter.show(jupyter_backend='ipygany')

## Plot av 2s og 2p-orbital:
Her plotter vi 2s-orbitalet og 2p-orbitalene. Fargene i plottet angir verdien på bølgefunksjonen og viser hvor bølgefunksjonen er positiv eller negativ.

In [ ]:
theta = np.linspace(0, np.pi, 100)  # 0 <= theta <= 180
phi = np.linspace(0, 2.0*np.pi, 100)  # 0 <= phi <= 360
# Lag et grid over alle mulige theta og phi-kombinasjoner:
theta, phi = np.meshgrid(theta, phi)
xyz = np.array(
    [
        np.sin(theta) * np.cos(phi),
        np.sin(theta) * np.sin(phi),
        np.cos(theta),
    ]
)
r = 3
plotter = pv.Plotter(notebook=True, shape=(1, 4), window_size=(1000, 250))
psi_2s = beregn_orbital(2, 0, 0, r, theta, phi)
psi_2pz = beregn_orbital(2, 1, 0, r, theta, phi)
psi_2py = beregn_orbital(2, 1, -1, r, theta, phi)
psi_2px = beregn_orbital(2, 1, 1, r, theta, phi)
merkelapper = ('2s', '2pz', '2py', '2px')
funksjoner = (psi_2s, psi_2pz, psi_2py, psi_2px)
for i, (navn, psi) in enumerate(zip(merkelapper, funksjoner)):
    plotter.subplot(0, i)
    plotter.add_text(f'{navn}', font_size=10, color='k')
    orbital = np.abs(psi)**2
    X, Y, Z = orbital * xyz
    grid = pv.StructuredGrid(X, Y, Z)
    plotter.add_mesh(grid, scalars=psi.T, show_scalar_bar=False)
plotter.show(jupyter_backend='static')

## Plot av sp3-hybridorbitaler:
Vi lager nå hybridorbitaler ved å kombinere atomorbitalene:

In [ ]:
sp3_1 = 0.5 * (psi_2s + psi_2px + psi_2py + psi_2pz)
plot_hjelp_orbital(r, theta, phi, sp3_1)

Vi kan lage tre lineærkombinasjoner til. La oss plotte alle 4:

In [ ]:
sp3_2 = 0.5 * (psi_2s + psi_2px - psi_2py - psi_2pz)
sp3_3 = 0.5 * (psi_2s - psi_2px + psi_2py - psi_2pz)
sp3_4 = 0.5 * (psi_2s - psi_2px - psi_2py + psi_2pz)

plotter = pv.Plotter(notebook=True, shape=(1, 4), window_size=(1000, 250))
merkelapper = ('sp3(1)', 'sp3(2)', 'sp3(3)', 'sp3(4)')
funksjoner = (sp3_1, sp3_2, sp3_3, sp3_4)
for i, (navn, psi) in enumerate(zip(merkelapper, funksjoner)):
    plotter.subplot(0, i)
    plotter.add_text(f'{navn}', font_size=10, color='k')
    orbital = np.abs(psi)**2
    X, Y, Z = orbital * xyz
    grid = pv.StructuredGrid(X, Y, Z)
    plotter.add_mesh(grid, scalars=psi.T, show_scalar_bar=False)
plotter.show(jupyter_backend='static')

Tilslutt, la oss plotte alle 4 i samme plott, dette burde få en tetraedisk form:

In [ ]:
xyz = np.array(
    [
        r * np.sin(theta) * np.cos(phi),
        r * np.sin(theta) * np.sin(phi),
        r * np.cos(theta),
    ]
)

plotter = pv.Plotter(notebook=True)
plotter.set_background('white')
for psi in funksjoner:
    orbital = np.abs(psi)**2
    X, Y, Z = orbital * xyz
    grid = pv.StructuredGrid(X, Y, Z)

    fortegn = 2. * (psi - psi.min()) / np.ptp(psi) - 1
    plotter.add_mesh(grid, scalars=fortegn.T, show_scalar_bar=True, clim=[-1, 1],
                     scalar_bar_args={'title': 'Skalert fortegn', 'color': 'k'})
plotter.view_yx()
plotter.show(jupyter_backend='ipygany')